In [4]:
# jack's car rental
import numpy as np
import sys
sys.path.append('C:\Program Files\python36\Lib\site-packages')
import gym
from gym import Env, spaces
from gym.envs.toy_text import discrete

In [9]:
# car rental enviroment
# distribution model is difficult, try to establish a sample model
# class CarRentalEnv(discrete.DiscreteEnv):
#     metadata = {'render.modes': []}
#     def __init__(self, max_cars, most_moveable_cars):
#         nS = max_cars ** 2
#         nA = most_moveable_cars * 2 + 1
#         self.shape = (max_cars, max_cars)
#         P = {}
        
#         for s in range(nS):
#             P[s] = { a:[] for a in range(nA)}
#             x, y = np.unravel_index(s, shape)
#             for borrow in range(0, x+1): # max rentable cars is x
#                 for ret_in_x in range(0, max_cars+1):
                    
#         # cal the state and reward for each place
#         # state can be changed by three parts, 1 rent, 2, return 3, cars that moved
#         isd = np.ones(nS)/ nS
#         super.__init__(nS, nA, P, isd)
        
#     def _render(self):
#         pass

def categorical_sample(prob_n, np_random):
    prob_n = np.asarray(prob_n)
    csprob_n = np.cumsum(prob_n)
    return (csprob_n > np_random.rand()).argmax()

class CarRentalEnv(Env):
    # action means take how many cars from place one to place two, -5 means take 5 cars from place two to place one
    def __init__(self, max_cars, max_removeable_cars, isd=None):
        self.max_cars = max_cars
        self.nS = max_cars ** 2
        self.nA = 2 * max_removeable_cars + 1
        if isd == None:
            self.isd = np.ones(self.nS) / self.nS
        self.max_removeable_cars = max_removeable_cars
        self.shape = (max_cars, max_cars)
        self.observation_space = spaces.Discrete(self.nS)
        self.action_space = spaces.Discrete(self.nA)

        self.seed()
        self.s = categorical_sample(self.isd, self.np_random)
        self.lastaction = None

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def _render(self):
        pass
    
    def reset(self):
        self.s = categorical_sample(self.isd, self.np_random)
        self.lastaction = None
        return self.s
    
    def step(self,a):
        x, y = np.unravel_index(self.s, self.shape)
        assert a - self.max_removeable_cars <= x, f"error, removed cars cannot larger than {x}"
        assert self.max_removeable_cars - a <= y, f"error, removed cars cannot larger than {y}"
        available_x = x - (a - self.max_removeable_cars)
        available_y = y + (a - self.max_removeable_cars)
        # sample rent cars and return cars
        rent_cars_in_x = np.random.poisson(3)
        return_cars_in_x = np.random.poisson(3)
        actual_rent_cars_in_x = min(rent_cars_in_x, available_x)
        newx = min(available_x - actual_rent_cars_in_x + return_cars_in_x, self.max_cars)

        rent_cars_in_y = np.random.poisson(4)
        return_cars_in_y = np.random.poisson(2)
        actual_rent_cars_in_y = min(rent_cars_in_y, available_y)
        newy = min(available_y - actual_rent_cars_in_y + rent_cars_in_y, self.max_cars)

        s = np.ravel_multi_index((newx, newy), self.shape)
        self.s = s
        self.lastaction = a
        rewards = 10 * (actual_rent_cars_in_x + actual_rent_cars_in_y) - 2 * abs(a - self.max_removeable_cars)
        return (s, rewards, False, '')

In [8]:
# policy iteration to solve the car rental problem
env = CarRentalEnv(20)
def policy_iter():
    pass